In [14]:
import sys
import os
# initialize to make sure project root is in the path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
print({sys.executable} )

{'/Users/Bo/Documents/LLM/smart_travel_advise/.venv/bin/python'}


In [5]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.tools import tool
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_transformers import Html2TextTransformer, BeautifulSoupTransformer
from langchain_openai.chat_models import ChatOpenAI
from adviser.config import OPENAI_API_KEY


In [4]:
class AdviceInput(BaseModel):
    region: str = Field(description="""
    Lower case continent name that the search country resides in.
    it should be in one of the following 6 regions:
    africa, americas, asia, europe, middle-east, pacific
    """)
    country: str = Field(description="""
    Lower case country full name that needs advice of trevel.
    if the country name have multiple words, connect the space with `-`.
    
    Examples: 
    1. usa -> united-states-america
    2. People's Republic of China -> china
    """)
    
@tool(args_schema=AdviceInput)
def advice_whether_safe_to_travel(region:str, country:str) -> str:
    """Check if it is safe to travel to the country of the region"""
    return "Exercise normal safety precautions"


/Users/Bo/Documents/LLM/smart_travel_advise/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'advice_whether_safe_to_travel',
 'description': 'Check if it is safe to travel to the country of the region',
 'parameters': {'properties': {'region': {'description': '\n    Lower case continent name that the search country resides in.\n    it should be in one of the following 6 regions:\n    africa, americas, asia, europe, middle-east, pacific\n    ',
    'type': 'string'},
   'country': {'description': "\n    Lower case country full name that needs advice of trevel.\n    if the country name have multiple words, connect the space with `-`.\n    \n    Examples: \n    1. usa -> united-states-america\n    2. People's Republic of China -> china\n    ",
    'type': 'string'}},
  'required': ['region', 'country'],
  'type': 'object'}}

In [6]:
model = ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18").bind(functions=[convert_to_openai_function(advice_whether_safe_to_travel)])

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a trip advier which informs whether it is safe to travel to certain country"),
    ("user", "{query}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [8]:
query = "I would like to travel to bnia-and-herzegovina. Is it safe?"

In [9]:
result = chain.invoke({"query": query})

In [10]:
result.tool_input['country']

'bosnia-and-herzegovina'

In [11]:
result.tool_input['region']

'europe'

In [15]:
url = f"https://www.smartraveller.gov.au/destinations/{result.tool_input['region']}/{result.tool_input['country']}"
html = WebBaseLoader(url).load()
transformer = BeautifulSoupTransformer()
docs_transformed = transformer.transform_documents(
    html, tags_to_extract=["p", "li", "div", "a"]
)

In [16]:
docs_transformed[0]

Document(metadata={'source': 'https://www.smartraveller.gov.au/destinations/europe/bosnia-and-herzegovina', 'title': 'Bosnia & Herzegovina Travel Advice & Safety | Smartraveller', 'description': 'Australian Government travel advice for Bosnia and Herzegovina. Exercise a high degree of caution. Travel advice level YELLOW. Understand the risks, safety, laws and contacts.', 'language': 'en'}, page_content='')

In [18]:
template = """ 
    Given the following inputs:
    
    {title}: The title of the webpage.
    {description}: A brief description of the webpage content.
    {query}: The question that the traveller asked about the country he is going to travel to.
    ===================================================================================================
    
    Your task is to assess whether it is safe to travel to the specified country and provide reasons for your assessment.
    Your response should include an overall score based on the following levels:
    
    Do Not Travel: Significant risks that make travel extremely dangerous.
    Reconsider Your Need to Travel: High risks that may require reconsideration of travel plans.
    Exercise a High Degree of Caution: Some risks that necessitate careful planning and precautions.
    Exercise Normal Safety Precautions: General safety precautions similar to those you would take in your home country.
    If the {title} returns "page not found," respond with: "There is no trip advisory based on current information."
    
    Example
    Inputs:
    
    {title}: "Indonesia Travel Advice & Safety | Smartraveller"
    {description}: "Australian Government travel advice for Indonesia. Exercise a high degree of caution. Travel advice level YELLOW. Understand the risks, safety, laws and contacts."
    {query}: "I would like to travel to Indonesia. Is it safe?"
    Output:
    
    Travel Safety Level: "Exercise a high degree of caution"
    Reasons:
    Recent political instability.
    High crime rates in certain areas.
    Ongoing health concerns such as outbreaks of diseases.
    """
prompt_template = PromptTemplate(
    input_variables=["title", "description", "query"],
    template=template)

prompt_2 = prompt_template.format(
    title=docs_transformed[0].metadata['title'],
    description=docs_transformed[0].metadata['description'],
    query=query
)

In [19]:
model_2 =  ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18")

In [20]:
output_parser = StrOutputParser()

In [21]:
answer = model_2.invoke(prompt_2)

In [25]:
print(output_parser.parse(answer).content)

Travel Safety Level: "Exercise a high degree of caution"  
Reasons:  
- The Australian Government advises exercising a high degree of caution when traveling to Bosnia and Herzegovina, indicating that there are some risks involved.  
- There may be political and social tensions in certain areas that could affect safety.  
- Travelers should be aware of local laws and customs, as well as potential health risks.  
- It is important to stay informed about the current situation and to take necessary precautions while traveling.
